In [ ]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
from mako.template import Template
import requests
from requests.compat import quote

from PIL import Image
from io import BytesIO
import re

commons_site = pb.Site("commons", "commons")

In [ ]:
from modules.wmtools import is_commons_file, get_hash

In [ ]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [ ]:
url = 'http://premsa.gencat.cat/pres_fsvp/AppJava/governacio-relacions-institucionals/notapremsavw/301618/ca/la-consellera-de-governacio-el-sindic-daran-signen-lacord-financament-2017-del-conselh-generau.do'
user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
headers = {'User-Agent' : user_agent}
excluded = []
categories = ['June 2017 in Catalonia',
             'Meritxell Borràs',
             'Carles Barrera Sánchez']
cat_string = '\n'.join(['[[Category:'+i+']]' for i in categories])

r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
pub_date=soup.find_all("span", attrs={"itemprop": "datePublished"})[0].get_text().strip().split(' ')[0].split('-')
pub_date.reverse()
pub_date='-'.join(pub_date)
pub_date

In [ ]:
title = soup.find_all("h1", class_="FW_headline")[0].get_text().strip().replace('  ', ' ')
#title = u""
title

In [ ]:
article_content = soup.find_all("div", class_="FW_article-content")[0].get_text().strip().split('\n')[0]
#article_content = u"El conseller d’Agricultura, Alimentació i Acció Rural, Joaquim Llena, acompanyat de la directora general d’Agricultura i Ramaderia, Rosa Cubel, i del director dels Serveis Territorials del DAR a Barcelona, Jaume Balagué, ha presidit, avui, l’acte de cloenda de l’Assemblea General Ordinària de la Federació Catalana d'Indústries de la Carn (FECIC), i el dinar posterior."
article_content

In [ ]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{ca|1=${description}}}
|date=${date}
|source=[${url} Nota de Premsa - ${title}]
|author=Generalitat de Catalunya
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-gencat}}

[[Category:Files uploaded by User:Discasto]]
${cat_string}"""

vars = {
    "url": url,
    "description": article_content,
    "date": pub_date,
    "title": title,
    "cat_string": cat_string
}
t = Template(template)
_text = t.render(**vars)
_text

In [ ]:
image_list = [{"url": image["href"].strip(), "name": image["title"].replace(':', ' -').replace('  ', ' ').strip()} for image in soup.find_all("a", class_="external") if '.jpg' in image['href'].lower()]
image_list

In [ ]:
used_names = []

for i, image in enumerate(image_list):
    # If the image is excluded, skip
    if i in excluded:
        print ("Image excluded. Skipping")
        continue
        
    # First, the image is downloaded and stored
    image_url = quote(image["url"].encode('utf-8'), ':/')
    image_name = image["name"].replace(':', ' -').replace('  ', ' ') + '.jpg'
    image_path = os.path.join(images_directory, image_name)
    try: 
        r = requests.get(image_url, headers=headers)
        image = Image.open(BytesIO(r.content))
        image.save(image_path)
    except :
        print ('Failed download. Skipping')
        continue

    # If the image is already in Commons, skip
    if is_commons_file(get_hash(image_path)) :
        print ("Image already in commons. Skipping")
        os.remove(image_path)
        continue

    # If the image name is already in commons, find a new name
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)
        
    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # image upload
    bot = UploadRobot(image_path,
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)